# Reference
[uproot documentation](https://uproot.readthedocs.io/en/latest/)

In [ ]:
import csv
interaction_dictionary = {}
with open('interactions.csv') as f:
    reader = csv.DictReader(f)
    for row in reader:
        key = int(row.pop('Idx'))
        interaction = row.pop('Interaction')
        interaction_dictionary[key] = interaction

In [ ]:
def make_sequential(obj):
    seq_events = np.zeros_like(obj.event)
    seq_events[0] = obj.event[0]
    seq_event = 0
    seq_files = np.zeros_like(obj.event)
    seq_files[0] = obj.event[0]
    seq_file = 0
    for i in range(1, len(obj.event)):
        if obj.event[i] != obj.event[i - 1]:
            seq_event += 1
        if obj.event[i] < obj.event[i - 1]:
            seq_file += 1
        seq_events[i] = seq_event
        seq_files[i] = seq_file
    obj.event = seq_events
    obj.file = seq_files

In [ ]:
import uproot, numpy as np

#Add a criteria for energy of each pfo
#Add a criteria for adc for each plane
#Add a criteria for true energy
#Add a criteria to view CaloHits in each plane


class MCValidation:
    def __init__(self, filename, treename):
        file = uproot.open(filename)
        tree = file[treename]
        self.event = tree['event'].array(library="np") #linear list of events
        self.orig_event = tree["event"].array(library="np") #actual event number
        self.file = np.zeros_like(self.event)
        self.mc_id = tree["mcId"].array(library="np") #number of mc particles in event
        self.mc_energy = tree["mcEnergy"].array(library="np") #pdg code of particles in each event
        self.mc_pdg = tree["mcPDG"].array(library="np") #pdg code of particles in each event
        self.mc_tier = tree["mcTier"].array(library="np") #Which tier each event is folded back to
        self.mc_nhits = tree["mcNHits"].array(library="np") #number of mc particles hits in event
        is_nu_int = tree["isNuInteration"].array(library="np") #not functioning
        is_cr_int = tree["isCosmicRay"].array(library="np") #not functioning
        is_tb_int = tree["isTestBeam"].array(library="np") #not functioning
        self.environment = np.full(is_nu_int.shape, "??") #not functioning
        self.environment[np.where(is_nu_int)] = "nu" #not functioning
        self.environment[np.where(is_cr_int)] = "tb" #not functioning
        self.environment[np.where(is_tb_int)] = "cr" #not functioning
        self.is_leading_lepton = tree["isLeadingLepton"].array(library="np")
        self.is_michel = tree["isMichel"].array(library="np")
        self.n_matches = tree["nMatches"].array(library="np")
        self.reco_id_list = tree["recoIdVector"].array(library="np")
        self.reco_nhits_list = tree["nRecoHitsVector"].array(library="np")
        self.shared_nhits_list = tree["nSharedHitsVector"].array(library="np")
        self.purity_adc_list = tree["purityAdcVector"].array(library="np")
        self.purity_list_u = tree["purityVectorU"].array(library="np")
        self.purity_list_v = tree["purityVectorV"].array(library="np")
        self.purity_list_w = tree["purityVectorW"].array(library="np")
        self.purity_adc_list_u = tree["purityAdcVectorU"].array(library="np")
        self.purity_adc_list_v = tree["purityAdcVectorV"].array(library="np")
        self.purity_adc_list_w = tree["purityAdcVectorW"].array(library="np")
        self.completeness_adc_list = tree["completenessAdcVector"].array(library="np")
        self.completeness_list_u = tree["completenessVectorU"].array(library="np")
        self.completeness_list_v = tree["completenessVectorV"].array(library="np")
        self.completeness_list_w = tree["completenessVectorW"].array(library="np")
        self.completeness_adc_list_u = tree["completenessAdcVectorU"].array(library="np")
        self.completeness_adc_list_v = tree["completenessAdcVectorV"].array(library="np")
        self.completeness_adc_list_w = tree["completenessAdcVectorW"].array(library="np")
        self.pc_metric = self.purity_adc_list * self.completeness_adc_list
        self.vtx_dx = tree["vtxDx"].array(library="np")
        self.vtx_x = tree["vtxX"].array(library="np")
        self.vtx_dy = tree["vtxDy"].array(library="np")
        self.vtx_y = tree["vtxY"].array(library="np")
        self.vtx_dz = tree["vtxDz"].array(library="np")
        self.vtx_z = tree["vtxZ"].array(library="np")
        self.vtx_dr = tree["vtxDr"].array(library="np")
        file.close()
        make_sequential(self)

In [ ]:
import os

def save_plot(fig, filename, subdir=None):
    if subdir is None:
        subdir = ""
    elif subdir.startswith("/"):
        subdir = subdir[1:]
        
    if not os.path.exists('images'):
        os.mkdir('images')
    for img_type in [ "png", "svg", "eps" ]:
        if not os.path.exists(f'images/{img_type}'):
            os.mkdir(f'images/{img_type}')
        if not os.path.exists(f'images/{img_type}/{subdir}'):
            os.mkdir(f'images/{img_type}/{subdir}')
        fig.savefig(f'images/{img_type}/{subdir}/{filename}.{img_type}', dpi=200)

# Reading data

In [ ]:
#For reading singular arrays

validation = MCValidation("hierarchy_validation_MC_test.root", "events")
x = validation.vtx_x
print(np.unique(x))
print(np.size(x))
print("Here:", np.min(x))
for i in range(0, np.size(x)):
    print(x[i])



In [ ]:
#For reading arrays with multiple entries

x = validation.mc_tier
print(np.size(x))
for i in range(0, np.size(x)):
    size = np.size(x[i])
    print(size)
    if size > 1:
        j = 0
        while j < size:
            print("here")
            print(x[i][j])
            j += 1
    else:
        print(x[i])
    

In [ ]:
# Printing vertex info
validation = MCValidation("hierarchy_validation_MC_22MeV.root", "events")
n_vectors = np.size(validation.vtx_dx)
for i in range(0, n_vectors):
    vertex_vector = []
    vertex_vector.append(validation.vtx_dx[i])
    vertex_vector.append(validation.vtx_dy[i])
    vertex_vector.append(validation.vtx_dz[i])
    vertex_vector.append(validation.vtx_dr[i])
    print(vertex_vector)

# Plotting coordinates of the vertices

In [ ]:
import numpy as np
cut = 0.03

print("\033[1m" "mcEnergy")


validation = MCValidation("hierarchy_validation_MC_test.root", "events")
x = validation.mc_energy
max_value = np.max(x)
bins = np.linspace(0, 0.03, 200)
idx = np.where(x < cut)
x[idx]
plt.hist(x[idx], density=False, bins=bins, histtype = 'step' )

plt.show()

In [ ]:
import numpy as np

print("\033[1m" "X Coordinate")

validation = MCValidation("hierarchy_validation_MC_test.root", "events")
x = validation.vtx_x
max_value = np.max(x)
bins = np.linspace(0, max_value, 100)
idx = np.where(x != 0)
x[idx]
plt.hist(x[idx], density=False, bins=bins, histtype = 'step' )

plt.show()

print("\033[1m" "Y Coordinate")

y = validation.vtx_y
max_value = np.max(y)
bins = np.linspace(0, max_value, 100)
idx = np.where(y != 0)
y[idx]
plt.hist(y[idx], density=False, bins=bins, histtype = 'step' )

plt.show()

print("\033[1m" "Z Coordinate")

z = validation.vtx_z
max_value = np.max(z)
bins = np.linspace(0, max_value, 100)
idx = np.where(z != 0)
z[idx]
plt.hist(z[idx], density=False, bins=bins, histtype = 'step' )


plt.show()


print("\033[1m" "Dx Coordinate")

validation = MCValidation("hierarchy_validation_MC_test.root", "events")
x = validation.vtx_dx
max_value = np.max(x)
bins = np.linspace(0, max_value, 100)
idx = np.where(x != 0)
x[idx]
plt.hist(x[idx], density=False, bins=bins, histtype = 'step' )

plt.show()

print("\033[1m" "Dy Coordinate")

y = validation.vtx_dy
max_value = np.max(y)
bins = np.linspace(0, max_value, 100)
idx = np.where(y != 0)
y[idx]
plt.hist(y[idx], density=False, bins=bins, histtype = 'step' )

plt.show()

print("\033[1m" "Dz Coordinate")

z = validation.vtx_dz
max_value = np.max(z)
bins = np.linspace(0, max_value, 100)
idx = np.where(z != 0)
z[idx]
plt.hist(z[idx], density=False, bins=bins, histtype = 'step' )


plt.show()


print("\033[1m" "Dr Coordinate")

z = validation.vtx_dr
max_value = np.max(z)
bins = np.linspace(0, max_value, 100)
idx = np.where(z != 0)
z[idx]
plt.hist(z[idx], density=False, bins=bins, histtype = 'step' )


plt.show()


# Plotting mc_nhits

In [ ]:
import statistics as st
import matplotlib.pyplot as plt
#All_energies = [5, 6, 7, 8, 9,13,14,16,17,20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
All_energies = list(range(5, 10)) + [13, 14, 16, 17] + list(range(20, 31))

def make_linear(validation, size_criteria):
    mc_pdg = MCValidation("hierarchy_validation_MC_" + str(file_energy) + "MeV.root", "events").mc_pdg
    temp_array_phot = []
    temp_array_elec = []
    for i in range(0, size_criteria):
        if mc_pdg[i] == 22: 
            subdivision_size = np.size(validation[i])
            x = 0
            empty = 0
            if subdivision_size == 0:
                empty += 1
            if subdivision_size == 1:
                if file_criteria == "mc_nhits" or "mc_pdg":
                    temp_array_phot.append(validation[i])
                else: 
                    temp_array_phot.append(validation[i][0])
            if subdivision_size > 1:
                while x < subdivision_size:
                    temp_array_phot.append(validation[i][x])
                    x += 1
        if mc_pdg[i] == 11: 
            subdivision_size = np.size(validation[i])
            x = 0
            empty = 0
            if suhttp://localhost:8888/notebooks/hierarchy.ipynb#Plotting-mc_nhitsbdivision_size == 0:
                empty += 1
            if subdivision_size == 1:
                if file_criteria == "mc_nhits" or "mc_pdg":
                    temp_array_elec.append(validation[i])
                else: 
                    temp_array_elec.append(validation[i][0])
            if subdivision_size > 1:
                while x < subdivision_size:
                    temp_array_elec.append(validation[i][x])
                    x += 1
    return temp_array_phot, temp_array_elec

#size_difference = int(make_linear(full_criteria)[1]) - int(size_criteria)
7717
#Compatible criteria "reco_nhits_list", "shared_nhits_list", "reco_id_list"

file_criteria = "mc_nhits" #From list of keys above
file_energy = 5
Loop_for_all_energies = True
print_array = False
plot = True

if Loop_for_all_energies == True:
    Energies = All_energies
else:
    Energies = [file_energy]
for i in Energies:
    file_energy = i #In MeV, which energy file should be read
    validation = MCValidation("hierarchy_validation_MC_" + str(file_energy) + "MeV.root", "events")
    full_criteria = eval("validation." + file_criteria)
    nice_array = make_linear(full_criteria, np.size(full_criteria))
    size_difference = np.size(nice_array[0]) + np.size(nice_array[1]) - np.size(full_criteria) 
    print("\033[1m" + file_criteria + "\033[0m" + " is as follows:")
    print("\n")
    print("The length of " + file_criteria + " is " + "\033[1m" + str(np.size(full_criteria)) + "\033[0m")
    print("The number of empty events recorded " + "\033[1m" + str(abs(size_difference)) + "\033[0m" )
    print("\n")
    print("The mean number of photon hits for", i, "MeV is", st.mean(nice_array[0]))
    print("The median number of photon hits for", i, "MeV is",st.median(nice_array[0]))
    print("\n")
    print("The mean number of electron hits for", i, "MeV is", st.mean(nice_array[1]))
    print("The median number of electron hits for", i, "MeV is",st.median(nice_array[1]))
    
    if print_array == True:
        print(nice_array)
    
    #Plotting
    if plot == True:
        #max_value = np.max(nice_array)
        bins = np.linspace(0, 100 , 100)
        #bins = np.linspace(0, max_value, max_value)
        plt.hist(nice_array[0], density=False, bins=bins, histtype = 'step', label = "photon")
        plt.hist(nice_array[1], density=False, bins=bins, histtype = 'step', label = "electron")
        plt.legend()
        #plt.xlim(3, np.max(nice_array))
        plt.show()
    
    

# Plotting reco_nhits_list

In [ ]:
import statistics as st
import matplotlib.pyplot as plt
#All_energies = [5, 6, 7, 8, 9,13,14,16,17,20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
All_energies = list(range(5, 10)) + [13, 14, 16, 17] + list(range(20, 31))

def make_linear(validation, file_criteria, size_criteria):
    temp_array = []
    #emp_array_elec = []
    for i in range(0, size_criteria):
        subdivision_size = np.size(validation[i])
        x = 0
        empty = 0
        if subdivision_size == 0:
            empty += 1
        if subdivision_size == 1:
            temp_array.append(validation[i][0])
        if subdivision_size > 1:
            while x < subdivision_size:
                temp_array.append(validation[i][x])
                x += 1
    return temp_array

#size_difference = int(make_linear(full_criteria)[1]) - int(size_criteria)

#Compatible criteria "reco_nhits_list", "shared_nhits_list", "reco_id_list"

file_criteria = "reco_nhits_list" #From list of keys above
file_energy = 5
Loop_for_all_energies = True
print_array = False
plot = True

if Loop_for_all_energies == True:
    Energies = All_energies
else:
    Energies = [file_energy]
for i in Energies:
    file_energy = i #In MeV, which energy file should be read
    validation = MCValidation("hierarchy_validation_MC_" + str(file_energy) + "MeV.root", "events")
    full_criteria = eval("validation." + file_criteria)
    #if full_criteria == "mc_nhits":
     #   nice_array = np.concatenate(full_criteria).flatten()
    #if full_criteria = "reco_nhits_list"
    nice_array = make_linear(full_criteria, file_criteria, np.size(full_criteria))
    size_difference = np.size(nice_array) - np.size(full_criteria) 
    print("\033[1m" + file_criteria + "\033[0m" + " is as follows:")
    print("\n")
    #print(np.min(nice_array))
    print("The length of " + file_criteria + " is " + "\033[1m" + str(np.size(full_criteria)) + "\033[0m")
    print("The number of empty events recorded " + "\033[1m" + str(abs(size_difference)) + "\033[0m" )
    #print("The mean number of reco hits for", i, "MeV is", st.mean(nice_array))
    #print("The median number of reco hits for", i, "MeV is",st.median(nice_array))
    
    if print_array == True:
        print(nice_array)
    
    #Plotting
    if plot == True:
        max_value = np.max(nice_array)
        bins = np.linspace(0, 100 , 100)
        #bins = np.linspace(0, max_value, max_value)
        plt.hist(nice_array, density=False, bins=bins, histtype = 'step' )
        #plt.xlim(3, np.max(nice_array))
        plt.show()
    
    

# Individual Efficiency

In [ ]:
import math
purity_cut = 0
completeness_cut = 0.65
Total_events = 5000

def get_efficiency(validation, pdg, signed=False, is_michel=False, tier=None):
    mc_pdg = abs(validation.mc_pdg) if not signed else validation.mc_pdg
    pdg = abs(pdg) if not signed else pdg
    particle_filter = mc_pdg == pdg
    if is_michel:
        particle_filter = particle_filter & (validation.is_michel)
    if tier:
        particle_filter = particle_filter & (validation.mc_tier == tier)

    mc_idx = np.where(particle_filter)

    #mc_purity = np.array([val[0] if len(val) > 0 else -1 for val in validation.purity_list])
    #mc_completeness = np.array([val[0] if len(val) > 0 else -1 for val in validation.completeness_list])
    # adjust this to just look at the quality tag
    #mc_good_match = np.where((particle_filter) & (validation.n_matches == 1) & \
    #                         (mc_purity > purity_cut) & \
    #                         (mc_completeness > completeness_cut))
    mc_good_match = np.where((particle_filter) & (validation.n_matches == 1))
    mc_no_match = np.where((particle_filter) & (validation.n_matches == 0))
    # adjust this to look at quality tag
    #mc_poor_match = np.where((particle_filter) & (validation.n_matches > 1))
    mc_poor_match = np.where((particle_filter) & (validation.n_matches > 0) & (validation.n_matches != 1))

    n_good_matches = len(mc_good_match[0])
    n_no_matches = len(mc_no_match[0])
    n_poor_matches = len(mc_poor_match[0])

    return np.array([n_good_matches, n_poor_matches, n_no_matches])

def get_all_efficiencies(validation, michel_tag=False, tier=None):
    photon_matches = get_efficiency(validation, 22, tier=tier)
    proton_matches = get_efficiency(validation, 2212, tier=tier)
    muon_matches = get_efficiency(validation, 13, tier=tier)
    electron_matches = get_efficiency(validation, 11, is_michel=michel_tag, tier=tier)
    pion_matches = get_efficiency(validation, 211, tier=tier)
    kaon_matches = get_efficiency(validation, 321, tier=tier)
    photon_eff = photon_matches / photon_matches.sum() if photon_matches.sum() else np.zeros_like(photon_matches)
    proton_eff = proton_matches / proton_matches.sum() if proton_matches.sum() else np.zeros_like(proton_matches)
    muon_eff = muon_matches / muon_matches.sum() if muon_matches.sum() else np.zeros_like(muon_matches)
    electron_eff = electron_matches / electron_matches.sum() if electron_matches.sum() else np.zeros_like(electron_matches)
    pion_eff = pion_matches / pion_matches.sum() if pion_matches.sum() else np.zeros_like(pion_matches)
    kaon_eff = kaon_matches / kaon_matches.sum() if kaon_matches.sum() else np.zeros_like(kaon_matches)
    electron_error = [math.sqrt(abs((electron_eff[0] * (electron_eff[0] - 1))/ electron_matches.sum()))]
    all_eff = np.concatenate((muon_eff[:,None], proton_eff[:,None], pion_eff[:,None], \
                                  kaon_eff[:,None], photon_eff[:,None], electron_eff[:,None],), \
                                axis=1)
    return all_eff

In [ ]:
labelsize=14
titlesize=18

import matplotlib as mpl
import matplotlib.pyplot as plt
from cycler import cycler

def plot_efficiencies(all_eff, file_prefix, tier=None):
    particle_labels = ['\u03bc', 'p', '\u03c0', '\u03ba', '\u03b3', 'e']
    metric_labels = ['Good match', 'Poor match', 'Unmatched']
    width = 0.5       # the width of the bars: can also be len(x) sequence

    fig, ax = plt.subplots(figsize=(12,8))

    # good
    ax.bar(particle_labels, all_eff[0], width, label=metric_labels[0], color='green')
    # poor
    ax.bar(particle_labels, all_eff[1], width, label=metric_labels[1], bottom=all_eff[0], color='orange')
    # unmatched
    ax.bar(particle_labels, all_eff[2], width, label=metric_labels[2], bottom=all_eff[0] + all_eff[1], color='red')

    ax.tick_params(axis='x', labelsize=labelsize)
    ax.tick_params(axis='y', labelsize=labelsize)
    ax.set_xlabel("particle", fontsize=titlesize)
    ax.set_ylabel('fraction', fontsize=titlesize)
    ax.set_title('Reconstruction efficiency', fontsize=titlesize)

    ax.legend(fontsize=titlesize)

    fig.tight_layout()
    plt.show()
    if tier is None:
        file_suffix = "all"
    else:
        file_suffix = f"tier_{tier}"
    save_plot(fig, f'{file_prefix}_{file_suffix}_eff', subdir="efficiencies")

In [ ]:
def run_eff(filename, treename, plot, michel_tag=False, tier=None):
    validation = MCValidation(filename, treename)
    all_eff = get_all_efficiencies(validation, michel_tag=michel_tag, tier=tier)
    file_prefix = ""
    if plot == True:
        print("Running: ", filename)
        print("Good Electrons:", all_eff[0][5])
        print("Poor Electrons:", all_eff[1][5])
        print("No Electrons:", all_eff[2][5])
        print("Good Photons:", all_eff[0][4])
        print("Poor Photons:", all_eff[1][4])
        print("No Photons:", all_eff[2][4])
        if "stm" in filename:
            file_prefix += "streamed"
        else:
            file_prefix += "standard"
        if "unfolded" in filename:
            file_prefix += "unfolded"
        else:
            file_prefix += "folded"
        plot_efficiencies(all_eff, file_prefix, tier)

In [ ]:
All_energies = [5, 6, 7, 8, 9, 13, 14, 16, 17, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]


def run_all(indx1, indx2, plot = False):
    xaxis_elec_eff = []
    yaxis_elec_eff = []
    for i in All_energies:
        filename = "hierarchy_validation_MC_" + str(i) + "MeV.root"
        treename = "events"
        validation = MCValidation(filename, treename)
        all_info = get_all_efficiencies(validation)
        run_eff(filename, treename, plot)
        xaxis_elec_eff.append(i)
        yaxis_elec_eff.append(all_info[indx1][indx2])
    return xaxis_elec_eff, yaxis_elec_eff
   

In [ ]:
print(run_all(0, 5, False))

# Combined Efficiency

In [ ]:
plt.tick_params(axis='x', labelsize=labelsize)
plt.tick_params(axis='y', labelsize=labelsize)
plt.xlabel("Monoenergetic energy sample MeV", fontsize=titlesize)
plt.ylabel('Fraction', fontsize=titlesize)
plt.title('Reconstruction efficiency', fontsize=titlesize)
plt.plot(run_all(0, 5, False)[0], run_all(0, 5, False)[1], label = "electron")
plt.plot(run_all(0, 4, False)[0], run_all(0, 4, False)[1], label = "photon")
# plt.plot(run_all(0, 1, False)[0], run_all(0, 1, False)[1], label = "proton")
plt.legend()
plt.tight_layout()
plt.show()

# Purity and Completeness

In [ ]:
def get_metric(metric_type, validation, pdg, leading_lepton=False, weighted=False, view=None, signed=False,
               is_michel=False, tier=None):
    mc_pdg = validation.mc_pdg # if not signed else validation.mc_pdg
    pdg = abs(pdg) if not signed else pdg
    particle_filter = mc_pdg == pdg
    if is_michel:
        particle_filter = particle_filter & (validation.is_michel)
    if tier:
        particle_filter = particle_filter & (validation.mc_tier == tier)
    
    if not leading_lepton:
        idx = np.where((particle_filter) & (validation.n_matches >= 1))
    else:
        idx = np.where((particle_filter) & (validation.n_matches >= 1) & (validation.is_leading_lepton))

    if not weighted:
        if metric_type == "purity":
            if not view:
                metric = validation.purity_list[idx]
            elif view.lower() == "u":
                metric = validation.purity_list_u[idx]
            elif view.lower() == "v":
                metric = validation.purity_list_v[idx]
            elif view.lower() == "w":
                metric = validation.purity_list_w[idx]
            else:
                raise Exception(f"Unknown view {view.upper()}")
        elif metric_type == "completeness":
            if not view:
                metric = validation.completeness_list[idx]
            elif view.lower() == "u":
                metric = validation.completeness_list_u[idx]
            elif view.lower() == "v":
                metric = validation.completeness_list_v[idx]
            elif view.lower() == "w":
                metric = validation.completeness_list_w[idx]
            else:
                raise Exception(f"Unknown view {view.upper()}")
        else:
            if not view:
                metric = validation.pc_metric[idx]
            else:
                raise Exception(f"Unknown view {view.upper()}")
    else:
        if metric_type == "purity":
            if not view:
                metric = validation.purity_adc_list[idx]
            elif view.lower() == "u":
                metric = validation.purity_adc_list_u[idx]
            elif view.lower() == "v":
                metric = validation.purity_adc_list_v[idx]
            elif view.lower() == "w":
                metric = validation.purity_adc_list_w[idx]
            else:
                raise Exception(f"Unknown view {view.upper()}")
        elif metric_type == "completeness":
            if not view:
                metric = validation.completeness_adc_list[idx]
            elif view.lower() == "u":
                metric = validation.completeness_adc_list_u[idx]
            elif view.lower() == "v":
                metric = validation.completeness_adc_list_v[idx]
            elif view.lower() == "w":
                metric = validation.completeness_adc_list_w[idx]
            else:
                raise Exception(f"Unknown view {view.upper()}")
        else:
            if not view:
                metric = validation.pc_metric[idx]
            else:
                raise Exception(f"Unknown view {view.upper()}")

    return metric.flatten()

def get_purity(validation, pdg, leading_lepton=False, weighted=False, view=None, signed=False,
               is_michel=False, tier=None):
    return get_metric("purity", validation, pdg, leading_lepton, weighted, view, signed, is_michel, tier)

def get_completeness(validation, pdg, leading_lepton=False, weighted=False, view=None, signed=False,
                     is_michel=False, tier=None):
    return get_metric("completeness", validation, pdg, leading_lepton, weighted, view, signed, is_michel, tier)

def get_pc_metric(validation, pdg, leading_lepton=False, weighted=False, view=None, signed=False,
                  is_michel=False, tier=None):
    return get_metric("pc_metric", validation, pdg, leading_lepton, weighted, None, signed, is_michel, tier)

def get_all_purities(validation, weighted=False, view=None, michel_tag=False, tier=None):
    muon_purity = get_purity(validation, 13, weighted=weighted, view=view, tier=tier)
    proton_purity = get_purity(validation, 2212, weighted=weighted, view=view, tier=tier)
    pion_purity = get_purity(validation, 211, weighted=weighted, view=view, tier=tier)
    kaon_purity = get_purity(validation, 321, weighted=weighted, view=view, tier=tier)
    photon_purity = get_purity(validation, 22, weighted=weighted, view=view, tier=tier)
    electron_purity = get_purity(validation, 11, weighted=weighted, view=view, is_michel=michel_tag, tier=tier)
    all_purity = [muon_purity, proton_purity, pion_purity, kaon_purity,
                                 photon_purity, electron_purity]
    return all_purity

def get_all_completenesses(validation, weighted=False, view=None, michel_tag=False, tier=None):
    muon_completeness = get_completeness(validation, 13, weighted=weighted, view=view, tier=tier)
    proton_completeness = get_completeness(validation, 2212, weighted=weighted, view=view, tier=tier)
    pion_completeness = get_completeness(validation, 211, weighted=weighted, view=view, tier=tier)
    kaon_completeness = get_completeness(validation, 321, weighted=weighted, view=view, tier=tier)
    photon_completeness = get_completeness(validation, 22, weighted=weighted, view=view, tier=tier)
    electron_completeness = get_completeness(validation, 11, weighted=weighted, view=view, is_michel=michel_tag, tier=tier)
    all_completeness = [muon_completeness, proton_completeness, pion_completeness,
                                       kaon_completeness, photon_completeness, electron_completeness]
    return all_completeness

def get_all_pc_metrics(validation, weighted=False, michel_tag=False, tier=None):
    muon_purity = get_pc_metric(validation, 13, weighted=weighted, view=None, tier=tier)
    proton_purity = get_pc_metric(validation, 2212, weighted=weighted, view=None, tier=tier)
    pion_purity = get_pc_metric(validation, 211, weighted=weighted, view=None, tier=tier)
    kaon_purity = get_pc_metric(validation, 321, weighted=weighted, view=None, tier=tier)
    photon_purity = get_pc_metric(validation, 22, weighted=weighted, view=None, tier=tier)
    electron_purity = get_pc_metric(validation, 11, weighted=weighted, view=None, is_michel=michel_tag, tier=tier)
    all_purity = [muon_purity, proton_purity, pion_purity, kaon_purity,
                                 photon_purity, electron_purity]
    return all_purity

def get_leading_lepton_purities(validation, weighted=False):
    muon_purity = get_purity(validation, 13, leading_lepton=True, weighted=weighted)
    electron_purity = get_purity(validation, 11, leading_lepton=True, weighted=weighted)
    all_purity = [muon_purity, electron_purity]
    return all_purity

def get_leading_lepton_completenesses(validation, weighted=False):
    muon_completeness = get_completeness(validation, 13, leading_lepton=True, weighted=weighted)
    electron_completeness = get_completeness(validation, 11, leading_lepton=True, weighted=weighted)
    all_completeness = [muon_completeness, electron_completeness]
    return all_completeness

def get_leading_lepton_pc_metric(validation, weighted=False):
    muon_purity = get_pc_metric(validation, 13, leading_lepton=True, weighted=weighted)
    electron_purity = get_pc_metric(validation, 11, leading_lepton=True, weighted=weighted)
    all_purity = [muon_purity, electron_purity]
    return all_purity

In [ ]:
print(validation.mc_pdg)
validation = MCValidation("hierarchy_validation_MC_16MeV.root", "events")
value1 = get_all_pc_metrics(validation)
print(value1)
print(value1[5])
print(len(value1[5]))
print(float(value1[5][80]))
value10 = []
for i in range(0,1316):
    value10.append(float(value1[5][i][0]))
    try:
        value10.append(float(value1[5][i][1]))
    except IndexError:
        continue
plot_metric(value10, value10, "PC", "all", "pc")

In [ ]:
def plot_metric(values1, values2, labels, particle, metric, leading_lepton=False, tier=None, weighted=False, view=None, log=True):
    if not leading_lepton:
        particle_labels = ['\u03bc', 'p', '\u03c0', '\u03ba', '\u03b3', 'e']
    else:
        particle_labels = ['\u03bc', 'e']

    fig, ax = plt.subplots(figsize=(12,8))
    
    n_bins = 20
    
    bins = np.linspace(0, 1, n_bins + 1)

    weights1 = np.ones_like(values1) / len(values1)
    ax.hist(values1, bins=bins, weights=weights1, histtype='step', label=labels[0])
    weights2 = np.ones_like(values2) / len(values2)
    ax.hist(values2, bins=bins, weights=weights2, histtype='step', label=labels[1])
    
    ax.tick_params(axis='x', labelsize=labelsize)
    ax.tick_params(axis='y', labelsize=labelsize)
    ax.set_xlabel(f"{metric}", fontsize=titlesize)
    ax.set_ylabel('event fraction', fontsize=titlesize)
    
    title = particle.title() + " " + metric.title()
    if view:
        title += f' {view.upper()}'
    if weighted:
        title += ' (ADC weighted)'
    ax.set_title(f'{title}', fontsize=titlesize)
    ax.legend(fontsize=titlesize, loc='upper left')

    if log:
        plt.yscale('log')
    limits = (ax.get_ylim()[0], 1)
    ax.set_ylim(limits)

    fig.tight_layout()
    plt.show()
    filename = f'{particle.lower()}_{metric.lower()}'
    if weighted:
        filename += "_weighted"
    if leading_lepton:
        filename += "_leading"
    if view:
        filename += f"_{view.lower()}"
    if tier is None:
        file_suffix = "all"
    else:
        file_suffix = f"tier_{tier}"
    save_plot(fig, f'{filename}_{file_suffix}', subdir=metric)

# Vertexing

In [ ]:
def get_node_vertex_delta(validation, percentile=68.27):
    return (np.percentile(np.abs(validation.vtx_dx), percentile),
            np.percentile(np.abs(validation.vtx_dy), percentile),
            np.percentile(np.abs(validation.vtx_dz), percentile),
            np.percentile(validation.vtx_dr, percentile))

In [ ]:
def plot_vertex_delta(streamed_delta, standard_delta, data_labels, delta_var, file_prefix):
    fig, ax = plt.subplots(figsize=(12,8))
    
    if delta_var.lower() != "dr":
        bins = np.linspace(-20.5, 20.5, 42)
    else:
        bins = np.linspace(0, 40, 81)
    std_weights = np.ones_like(standard_delta)
    std_weights /= len(standard_delta)
    stm_weights = np.ones_like(streamed_delta)
    stm_weights /= len(streamed_delta)
    
    ax.hist(streamed_delta, bins=bins, weights=stm_weights, histtype='step', label=data_labels[0])
    ax.hist(standard_delta, bins=bins, weights=std_weights, histtype='step', label=data_labels[1])
        
    ax.tick_params(axis='x', labelsize=labelsize)
    ax.tick_params(axis='y', labelsize=labelsize)
    ax.set_xlabel("reco - true", fontsize=titlesize)
    ax.set_ylabel('fraction', fontsize=titlesize)
    ax.set_title(f'Vertex {delta_var.lower()}', fontsize=titlesize)

    ax.legend(fontsize=titlesize)

    fig.tight_layout()
    plt.show()
    save_plot(fig, f'{file_prefix.lower()}_vertex_{delta_var.lower()}', subdir="vertex")

# Track/Shower Id

In [ ]:
def get_pid(validation, pdg, signed=False):
    mc_pdg = abs(validation.mc_pdg) if not signed else validation.mc_pdg
    pdg = abs(pdg) if not signed else pdg
    correct_pid = 11 if pdg in [11, -11, 22] else 13
    #mc_purity = np.array([val[0] if len(val) > 0 else -1 for val in validation.purity_list])
    #mc_completeness = np.array([val[0] if len(val) > 0 else -1 for val in validation.completeness_list])
    #mc_idx = np.where(mc_pdg == pdg)
    #mc_idx = np.where((mc_pdg == pdg) & (validation.n_matches == 1) & (mc_purity > purity_cut) & \
    #                  (mc_completeness > completeness_cut))
    mc_idx = np.where((mc_pdg == pdg) & (validation.n_matches > 0) & (validation.is_quality))   
    good_pids = validation.reco_id_list[mc_idx].flatten()
    return (good_pids == correct_pid).sum() / len(good_pids) if len(good_pids) else 0

def get_all_pid(validation):
    photon_pid = get_pid(validation, 22)
    proton_pid = get_pid(validation, 2212)
    muon_pid = get_pid(validation, 13)
    electron_pid = get_pid(validation, 11)
    pion_pid = get_pid(validation, 211)
    kaon_pid = get_pid(validation, 321)
    all_pid = np.array([muon_pid, proton_pid, pion_pid, kaon_pid, photon_pid, electron_pid])
    return all_pid

def plot_pids(all_pids, data_labels, file_prefix):
    particle_labels = ['\u03bc', 'p', '\u03c0', '\u03ba', '\u03b3', 'e']
    width = 0.5       # the width of the bars: can also be len(x) sequence
    colors = ['blue', 'orange']

    fig, ax = plt.subplots(figsize=(12,8))

    for i, all_pid in enumerate(all_pids):
        ax.bar(particle_labels, all_pid, width, label=data_labels[i], fill=False, edgecolor=colors[i])

    ax.tick_params(axis='x', labelsize=labelsize)
    ax.tick_params(axis='y', labelsize=labelsize)
    ax.set_xlabel("particle", fontsize=titlesize)
    ax.set_ylabel('fraction', fontsize=titlesize)
    ax.set_title('Correct PID', fontsize=titlesize)

    ax.legend(fontsize=titlesize, loc='upper center')

    fig.tight_layout()
    plt.show()
    save_plot(fig, f'{file_prefix}_pid', subdir="pid")

In [ ]:
def run_pid(filenames, labels, treename):
    all_pids = []
    for filename in filenames:
        validation = MCValidation(filename, treename)
        all_pids.append(get_all_pid(validation))
    file_prefix = ""
    if "unfolded" in filename:
        file_prefix += "unfolded"
    else:
        file_prefix += "folded"
    plot_pids(all_pids, labels, file_prefix)

In [ ]:
def run_vertex(filenames, labels, treename):
    all_vtx = []
    for filename in filenames:
        validation = MCValidation(filename, treename)
        all_vtx.append(validation)

    for i in range(len(labels)):
        vtx_deltas_68 = get_node_vertex_delta(all_vtx[i])
        print(f"{labels[i]} node vertex deltas: x = {vtx_deltas_68[0]:.2} y = {vtx_deltas_68[1]:.2} z = {vtx_deltas_68[2]:.2} r = {vtx_deltas_68[3]:.2}")
    
    plot_vertex_delta(all_vtx[0].vtx_dx, all_vtx[1].vtx_dx, labels, "dx", "node")
    plot_vertex_delta(all_vtx[0].vtx_dy, all_vtx[1].vtx_dy, labels, "dy", "node")
    plot_vertex_delta(all_vtx[0].vtx_dz, all_vtx[1].vtx_dz, labels, "dz", "node")
    plot_vertex_delta(all_vtx[0].vtx_dr, all_vtx[1].vtx_dr, labels, "dr", "node")

# Hierarchy

In [ ]:
def get_hierarchy_efficiency(validation, pdg, tier, signed=False):
    mc_pdg = abs(validation.mc_pdg) if not signed else validation.mc_pdg
    pdg = abs(pdg) if not signed else pdg
    particle_filter = mc_pdg == pdg
    particle_filter = particle_filter & (validation.mc_tier == tier)
    particle_filter = particle_filter & (validation.n_expected_children > 0)
    particle_filter = particle_filter & (validation.expected_children_adc_list.sum() > 0)

    mc_idx = np.where((particle_filter) & (validation.is_good_match))
    
    n_reco_children = validation.good_reco_children_list[mc_idx]
    adc_exp_children = validation.expected_children_adc_list[mc_idx]
    numerator = (n_reco_children * adc_exp_children).sum()
    denomenator = adc_exp_children.sum()

    return numerator / denomenator

In [ ]:
def plot_hierarchy_efficiency(efficiency_1, efficiency_2, pdg, data_labels):
    particle_labels = {13: '\u03bc', 2212: 'p', 211: '\u03c0', 321: '\u03ba', 22: '\u03b3', 11: 'e'}
    file_labels = {13: 'muon', 2212: 'proton', 211: 'pion', 321: 'kaon', 22: 'gamma', 11: 'electron'}
    colors = ['blue', 'orange']

    fig, ax = plt.subplots(figsize=(12,8))

    bins = np.linspace(0, 1, 21)
    weights1 = np.ones_like(efficiency_1) / len(efficiency_1)
    weights2 = np.ones_like(efficiency_2) / len(efficiency_2)
    
    ax.hist(efficiency_1, bins=bins, weights=weights1, label=data_labels[0], edgecolor=colors[0], histtype='step')
    ax.hist(efficiency_2, bins=bins, weights=weights2, label=data_labels[1], edgecolor=colors[1], histtype='step')

    ax.tick_params(axis='x', labelsize=labelsize)
    ax.tick_params(axis='y', labelsize=labelsize)
    ax.set_xlabel("efficiency", fontsize=titlesize)
    ax.set_ylabel('fraction', fontsize=titlesize)
    ax.set_title(f'{particle_labels[pdg]} hierarchy efficiency', fontsize=titlesize)

    ax.legend(fontsize=titlesize)

    fig.tight_layout()
    plt.show()
    save_plot(fig, f'hier_eff_{file_labels[pdg]}')

# Node level execution

In [ ]:
val_streamed = MCValidation("stm/hierarchy_validation_mc.root", "mc")
val_standard = MCValidation("std/hierarchy_validation_mc.root", "mc")

In [ ]:
len(np.unique(val_standard.event))

In [ ]:
len(np.unique(val_streamed.event))

In [ ]:
original_cycler = mpl.rcParams['axes.prop_cycle']
mpl.rcParams['axes.prop_cycle'] = cycler(color=['#2ca02c', '#1f77b4', '#ff7f0e', '#d62728', '#9467bd',
                                                '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])

for tier in [ None, 1 ]:
    run_eff("std/hierarchy_validation_mc.root", "mc", tier=tier)
    run_eff("stm/hierarchy_validation_mc.root", "mc", tier=tier)

mpl.rcParams['axes.prop_cycle'] = original_cycler

In [ ]:
for tier in [ None, 1 ]:
    ps_streamed = get_all_purities(val_streamed, tier=tier)
    ps_standard = get_all_purities(val_standard, tier=tier)
    cs_streamed = get_all_completenesses(val_streamed, tier=tier)
    cs_standard = get_all_completenesses(val_standard, tier=tier)
    pcs_streamed = get_all_pc_metrics(val_streamed, tier=tier)
    pcs_standard = get_all_pc_metrics(val_standard, tier=tier)
    for p, particle in enumerate(['muon', 'proton', 'pion', 'kaon', 'photon', 'electron']):
        plot_metric(ps_streamed[p], ps_standard[p], ["streamed", "standard"], particle=particle, metric="purity", tier=tier, log=False)
        plot_metric(cs_streamed[p], cs_standard[p], ["streamed", "standard"], particle=particle, metric="completeness", tier=tier, log=False)
        plot_metric(pcs_streamed[p], pcs_standard[p], ["streamed", "standard"], particle=particle, metric="purity_completeness", tier=tier, log=False)

In [ ]:
for tier in [ None, 1 ]:
    ps_streamed = get_all_purities(val_streamed, weighted=True, tier=tier)
    ps_standard = get_all_purities(val_standard, weighted=True, tier=tier)
    cs_streamed = get_all_completenesses(val_streamed, weighted=True, tier=tier)
    cs_standard = get_all_completenesses(val_standard, weighted=True, tier=tier)
    pcs_streamed = get_all_pc_metrics(val_streamed, weighted=True, tier=tier)
    pcs_standard = get_all_pc_metrics(val_standard, weighted=True, tier=tier)
    for p, particle in enumerate(['muon', 'proton', 'pion', 'kaon', 'photon', 'electron']):
        plot_metric(ps_streamed[p], ps_standard[p], ["streamed", "standard"], particle=particle, metric="purity", weighted=True, tier=tier, log=False)
        plot_metric(cs_streamed[p], cs_standard[p], ["streamed", "standard"], particle=particle, metric="completeness", weighted=True, tier=tier, log=False)
        plot_metric(pcs_streamed[p], pcs_standard[p], ["streamed", "standard"], particle=particle, metric="purity_completeness", weighted=True, tier=tier, log=False)

In [ ]:
filenames = ["stm/hierarchy_validation_mc.root", "std/hierarchy_validation_mc.root"]
labels = ["streamed", "standard"]
run_pid(filenames, labels, "mc")

In [ ]:
filenames = ["stm/hierarchy_validation_mc.root", "std/hierarchy_validation_mc.root"]
labels = ["streamed", "standard"]
run_vertex(filenames, labels, "mc")

# Hierarchy efficiency

Not currently implemented

In [ ]:
#pdg = 2212
#hier_eff_streamed = get_hierarchy_efficiency(val_streamed, pdg, 1)
#hier_eff_standard = get_hierarchy_efficiency(val_standard, pdg, 1)
#plot_hierarchy_efficiency(hier_eff_streamed, hier_eff_standard, pdg, ["streamed", "standard"])

# Event-level validation

In [ ]:
class EventValidation:
    def __init__(self, filename, treename):
        file = uproot.open(filename)
        tree = file[treename]
        self.event = tree.array("event")
        self.orig_event = tree.array("event")
        self.file = np.zeros_like(self.event)
        self.interaction_type = tree.array("interactionType")
        self.n_good_matches = tree.array("nGoodMatches")
        self.n_above_threshold_matches = tree.array("nPoorMatches")
        self.n_unmatched = tree.array("nUnmatched")
        self.n_nodes = tree.array("nNodes")
        self.n_good_tier1_matches = tree.array("nGoodTier1Matches")
        self.n_tier1_nodes = tree.array("nTier1Nodes")
        self.n_good_track_matches = tree.array("nGoodTrackMatches")
        self.n_track_nodes = tree.array("nTrackNodes")
        self.n_good_tier1_track_matches = tree.array("nGoodTier1TrackMatches")
        self.n_tier1_track_nodes = tree.array("nTier1TrackNodes")        
        self.n_good_shower_matches = tree.array("nGoodShowerMatches")
        self.n_shower_nodes = tree.array("nShowerNodes")
        self.n_good_tier1_shower_matches = tree.array("nGoodTier1ShowerMatches")
        self.n_tier1_shower_nodes = tree.array("nTier1ShowerNodes")        
        self.has_leading_muon = tree.array("hasLeadingMuon")
        self.has_leading_electron = tree.array("hasLeadingElectron")
        self.is_leading_lepton_correct = tree.array("isLeadingLeptonCorrect")
        self.nu_vtx_dx = tree.array("vtxDx")
        self.nu_vtx_dy = tree.array("vtxDy")
        self.nu_vtx_dz = tree.array("vtxDz")
        self.nu_vtx_dr = tree.array("vtxDr")
        file.close()
        make_sequential(self)

In [ ]:
def get_correct_event_fraction(validation, fltr=None):
    if not fltr:
        idx = np.where(validation.n_nodes > 0)
    else:
        idx = np.where((validation.n_nodes > 0) & (np.in1d(validation.interaction_type, fltr)))
    good = (validation.n_good_matches[idx] == validation.n_nodes[idx]).sum(dtype=np.float)
    n = (validation.n_nodes[idx] > 0).sum(dtype=np.float)
    err = np.sqrt((good * (n - good)) / (n**3))
    f = good / n
    return f, err

def get_event_correctness(validation):
    idx = np.where(validation.n_nodes > 0)
    return validation.n_good_matches[idx] / validation.n_nodes[idx]

def get_correct_event_fraction_by_n_primaries(validation):
    unique_node_count = np.unique(validation.n_nodes)
    correct_fractions = {}
    for node_count in unique_node_count:
        idx = np.where(validation.n_nodes == node_count)
        correct_fractions[node_count] = (validation.n_good_matches[idx] == validation.n_nodes[idx]).mean()
    
    return correct_fractions

def get_correct_tier1_fraction(validation, fltr=None):
    if not fltr:
        idx = np.where(validation.n_tier1_nodes > 0)
    else:
        idx = np.where((validation.n_tier1_nodes > 0) & (np.in1d(validation.interaction_type, fltr)))
    good = (validation.n_good_tier1_matches[idx] == validation.n_tier1_nodes[idx]).sum(dtype=np.float)
    n = (validation.n_tier1_nodes[idx] > 0).sum(dtype=np.float)
    err = np.sqrt(good * (n - good) / n**3)
    f = good / n
    return f, err

def get_tier1_correctness(validation):
    idx = np.where(validation.n_tier1_nodes > 0)
    return validation.n_good_tier1_matches[idx] / validation.n_tier1_nodes[idx]

def get_correct_leading_lepton_fraction(validation):
    if np.any(validation.has_leading_muon) or np.any(validation.has_leading_electron):
        correct = validation.is_leading_lepton_correct.sum()
        n = validation.has_leading_muon.sum() + validation.has_leading_electron.sum()
        f = correct / n
        err = np.sqrt(f) / n
        return f, err
    else:
        return 0., 0.

def get_correct_leading_muon_fraction(validation):
    if np.any(validation.has_leading_muon):
        correct = validation.is_leading_lepton_correct[np.where(validation.has_leading_muon)].sum()
        n = validation.has_leading_muon.sum()
        f = correct / n
        err = np.sqrt(f) / n
        return f, err
    else:
        return 0., 0.

def get_correct_leading_electron_fraction(validation):
    if np.any(validation.has_leading_electron):
        correct = validation.is_leading_lepton_correct[np.where(validation.has_leading_electron)].sum()
        n = validation.has_leading_electron.sum()
        f = correct / n
        err = np.sqrt(f) / n
        return f, err
    else:
        return 0., 0.

def get_above_threshold_event_fraction(validation):
    return ((validation.n_good_matches + validation.n_above_threshold_matches) == validation.n_nodes).sum() \
        / (validation.n_nodes > 0).sum()

def get_above_threshold_event_fraction_by_n_primaries(validation):
    unique_node_count = np.unique(validation.n_nodes)
    correct_fractions = {}
    for node_count in unique_node_count:
        idx = np.where(validation.n_nodes == node_count)
        correct_fractions[node_count] = ((validation.n_good_matches[idx] + validation.n_above_threshold_matches[idx]) \
                                         == validation.n_nodes[idx]).mean()
    
    return correct_fractions

def get_correct_tracks_fraction(validation):
    idx = np.where(validation.n_track_nodes > 0)
    return (validation.n_good_track_matches[idx] == validation.n_track_nodes[idx]).sum() / (validation.n_track_nodes > 0).sum()

def get_correct_tracks_fraction_by_n_tracks(validation):
    unique_node_count = np.unique(validation.n_track_nodes)
    correct_fractions = {}
    for node_count in unique_node_count:
        if not node_count: continue
        idx = np.where(validation.n_track_nodes == node_count)
        correct_fractions[node_count] = (validation.n_good_track_matches[idx] == validation.n_track_nodes[idx]).mean()
    
    return correct_fractions

def get_fraction_tracks_matched(validation):
    idx = np.where(validation.n_track_nodes > 0)
    return (validation.n_good_track_matches[idx] / validation.n_track_nodes[idx]).mean()

def get_correct_tier1_tracks_fraction(validation):
    idx = np.where(validation.n_tier1_track_nodes > 0)
    return (validation.n_good_tier1_track_matches[idx] == validation.n_tier1_track_nodes[idx]).sum() / (validation.n_tier1_track_nodes > 0).sum()

def get_fraction_tier1_tracks_matched(validation):
    idx = np.where(validation.n_tier1_track_nodes > 0)
    return (validation.n_good_tier1_track_matches[idx] / validation.n_tier1_track_nodes[idx]).mean()

def get_correct_showers_fraction(validation):
    idx = np.where(validation.n_shower_nodes > 0)
    return (validation.n_good_shower_matches[idx] == validation.n_shower_nodes[idx]).sum() / (validation.n_shower_nodes > 0).sum()

def get_correct_showers_fraction_by_n_showers(validation):
    unique_node_count = np.unique(validation.n_shower_nodes)
    correct_fractions = {}
    for node_count in unique_node_count:
        if not node_count: continue
        idx = np.where(validation.n_shower_nodes == node_count)
        correct_fractions[node_count] = (validation.n_good_shower_matches[idx] == validation.n_shower_nodes[idx]).mean()
    
    return correct_fractions

def get_correct_tier1_showers_fraction(validation):
    idx = np.where(validation.n_tier1_shower_nodes > 0)
    return (validation.n_good_tier1_shower_matches[idx] == validation.n_tier1_shower_nodes[idx]).sum() / (validation.n_tier1_shower_nodes > 0).sum()

def get_fraction_showers_matched(validation):
    idx = np.where(validation.n_shower_nodes > 0)
    return (validation.n_good_shower_matches[idx] / validation.n_shower_nodes[idx]).mean()

def get_fraction_tier1_showers_matched(validation):
    idx = np.where(validation.n_tier1_shower_nodes > 0)
    return (validation.n_good_tier1_shower_matches[idx] / validation.n_tier1_shower_nodes[idx]).mean()

def get_fraction_nodes_matched(validation):
    idx = np.where(validation.n_nodes > 0)
    return (validation.n_good_matches[idx] / validation.n_nodes[idx]).mean()

def get_fraction_nodes_matched_by_n_primaries(validation):
    idx = np.where(validation.n_nodes > 0)
    unique_node_count = np.unique(validation.n_nodes[idx])
    fraction_matched = {}
    for node_count in unique_node_count:
        idx = np.where(validation.n_nodes == node_count)
        fraction_matched[node_count] = (validation.n_good_matches[idx] / validation.n_nodes[idx]).mean()
    
    return fraction_matched

def get_neutrino_vertex_delta(validation, percentile=68.27):
    return (np.percentile(np.abs(validation.nu_vtx_dx), percentile),
            np.percentile(np.abs(validation.nu_vtx_dy), percentile),
            np.percentile(np.abs(validation.nu_vtx_dz), percentile),
            np.percentile(validation.nu_vtx_dr, percentile))

In [ ]:
def plot_event_correctness(streamed_correctness, standard_correctness, data_labels, file_prefix):
    fig, ax = plt.subplots(figsize=(12,8))

    bins = np.linspace(0, 1, 21)
    std_weights = np.ones_like(standard_correctness)
    std_weights /= len(standard_correctness)
    stm_weights = np.ones_like(streamed_correctness)
    stm_weights /= len(streamed_correctness)
    
    ax.hist(standard_correctness, bins=bins, weights=std_weights, histtype='step', label=data_labels[0])
    ax.hist(streamed_correctness, bins=bins, weights=stm_weights, histtype='step', label=data_labels[1])
    
    ax.tick_params(axis='x', labelsize=labelsize)
    ax.tick_params(axis='y', labelsize=labelsize)
    ax.set_xlabel("correct primaries fraction", fontsize=titlesize)
    ax.set_ylabel('fraction', fontsize=titlesize)
    ax.set_title('Correct Primaries Fraction', fontsize=titlesize)

    ax.legend(fontsize=titlesize, loc='upper left')

    fig.tight_layout()
    plt.show()
    save_plot(fig, f'{file_prefix}_event_correctness', subdir="event")

def plot_event_correctness_with_errors(streamed_correctness, standard_correctness, data_labels, file_prefix):
    fig, ax = plt.subplots(figsize=(12,8))

    bins = np.linspace(0, 1, 21)
    weights = np.ones_like(streamed_correctness)
    weights /= len(streamed_correctness)
    data1, _ = np.histogram(streamed_correctness, bins=bins)
    data2, _ = np.histogram(standard_correctness, bins=bins)

    bin_centres = 0.5*(bins[1:] + bins[:-1])
    err1 = np.sqrt(data1) / len(streamed_correctness)
    err2 = np.sqrt(data2) / len(standard_correctness)
    data1 = data1 / len(streamed_correctness)
    data2 = data2 / len(standard_correctness)
    width = 0.05
    
    ax.errorbar(bin_centres, data1, yerr=err1, marker='.', drawstyle='steps-mid', label=data_labels[0])
    ax.errorbar(bin_centres, data2, yerr=err2, marker='.', drawstyle='steps-mid', label=data_labels[1])
    
    ax.tick_params(axis='x', labelsize=labelsize)
    ax.tick_params(axis='y', labelsize=labelsize)
    ax.set_xlabel("Correct primaries fraction", fontsize=titlesize)
    ax.set_ylabel('Event fraction', fontsize=titlesize)
    ax.set_title('Correct Primaries Fraction', fontsize=titlesize)

    ax.legend(fontsize=titlesize)

    fig.tight_layout()
    plt.show()
    save_plot(fig, f'{file_prefix}_event_correctness')

In [ ]:
interaction_dictionary

In [ ]:
def run_event_level(filename, treename):
    validation = EventValidation(filename, treename)
    # nodes
    int_type_fltr = None#[200]
    correct_event_fraction, cef_err = get_correct_event_fraction(validation, fltr=int_type_fltr)
    correct_tier1_fraction, ctf_err = get_correct_tier1_fraction(validation, fltr=int_type_fltr)
    correct_leading_lepton_fraction, cllf_err = get_correct_leading_lepton_fraction(validation)
    correct_leading_muon_fraction, clmf_err = get_correct_leading_muon_fraction(validation)
    correct_leading_electron_fraction, clef_err = get_correct_leading_electron_fraction(validation)
    correct_event_fraction_by_n_primaries = get_correct_event_fraction_by_n_primaries(validation)
    correct_event_fraction_by_n_primaries = {k: correct_event_fraction_by_n_primaries[k] for
                                             k in correct_event_fraction_by_n_primaries if k < 16}
    almost_correct_fraction = get_above_threshold_event_fraction(validation)
    node_fraction = get_fraction_nodes_matched(validation)
    node_fraction_by_n_primaries = get_fraction_nodes_matched_by_n_primaries(validation)
    node_fraction_by_n_primaries = {k: node_fraction_by_n_primaries[k] for k in node_fraction_by_n_primaries if k < 16}
    print(f"Correct event fraction: {correct_event_fraction:.3} +- {cef_err:.1}")
    print(f"Correct tier 1 fraction: {correct_tier1_fraction:.3} +- {ctf_err:.1}")
    print(f"Correct leading lepton fraction: {correct_leading_lepton_fraction:.2} +- {cllf_err:.1} mu: {correct_leading_muon_fraction:.2} +- {clmf_err:.1} e: {correct_leading_electron_fraction:.2} +- {clef_err:.1}")
    print(f"Correct event fraction by N primaries: ", end="")
    for n_primaries in correct_event_fraction_by_n_primaries:
        print(f"{n_primaries}: {correct_event_fraction_by_n_primaries[n_primaries]:.2}", end=", ")
    print()
    print(f"Almost correct event fraction: {almost_correct_fraction:.2}")
    print(f"Fraction of nodes reconstructed: {node_fraction:.2}")
    print(f"Fraction of nodes reconstructed by N primaries: ")
    for n_primaries in node_fraction_by_n_primaries:
        print(f"{n_primaries}: {node_fraction_by_n_primaries[n_primaries]:.2}", end=", ")
    print()
    
    # tracks
    correct_tracks_fraction = get_correct_tracks_fraction(validation)
    correct_tier1_tracks_fraction = get_correct_tier1_tracks_fraction(validation)
    correct_tracks_fraction_by_n_primaries = get_correct_tracks_fraction_by_n_tracks(validation)
    correct_tracks_fraction_by_n_primaries = {k: correct_tracks_fraction_by_n_primaries[k] for
                                              k in correct_tracks_fraction_by_n_primaries if k < 16}
    track_fraction = get_fraction_tracks_matched(validation)
    tier1_track_fraction = get_fraction_tier1_tracks_matched(validation)
    print(f"Correct tracks fraction: {correct_tracks_fraction:.2}")
    print(f"Correct tier1 tracks fraction: {correct_tier1_tracks_fraction:.2}")
    print(f"Correct tracks fraction by N primaries: ")
    for n_primaries in correct_tracks_fraction_by_n_primaries:
        print(f"{n_primaries}: {correct_tracks_fraction_by_n_primaries[n_primaries]:.2}", end=", ")
    print()
    print(f"Fraction of tracks reconstructed: {track_fraction:.2}")
    print(f"Fraction of tier1 tracks reconstructed: {tier1_track_fraction:.2}")
    
    # showers
    correct_showers_fraction = get_correct_showers_fraction(validation)
    correct_tier1_showers_fraction = get_correct_tier1_showers_fraction(validation)
    correct_showers_fraction_by_n_primaries = get_correct_showers_fraction_by_n_showers(validation)
    correct_showers_fraction_by_n_primaries = {k: correct_showers_fraction_by_n_primaries[k] for
                                               k in correct_showers_fraction_by_n_primaries if k < 16}
    shower_fraction = get_fraction_showers_matched(validation)
    tier1_shower_fraction = get_fraction_tier1_showers_matched(validation)
    print(f"Correct showers fraction: {correct_showers_fraction:.2}")
    print(f"Correct tier1 showers fraction: {correct_tier1_showers_fraction:.2}")
    print(f"Correct showers fraction by N primaries: ")
    for n_primaries in correct_showers_fraction_by_n_primaries:
        print(f"{n_primaries}: {correct_showers_fraction_by_n_primaries[n_primaries]:.2}", end=", ")
    print()
    print(f"Fraction of showers reconstructed: {shower_fraction:.2}")
    print(f"Fraction of tier1 showers reconstructed: {tier1_shower_fraction:.2}")
    
    vtx_deltas_68 = get_neutrino_vertex_delta(validation)
    print(f"nu vertex deltas: x = {vtx_deltas_68[0]:.2} y = {vtx_deltas_68[1]:.2} z = {vtx_deltas_68[2]:.2} r = {vtx_deltas_68[3]:.2}")

In [ ]:
print("Standard")
run_event_level("std/hierarchy_validation_event.root", "events")
print("Streamed")
run_event_level("stm/hierarchy_validation_event.root", "events")

In [ ]:
stm_evt_validation = EventValidation("stm/hierarchy_validation_event.root", "events")
std_evt_validation = EventValidation("std/hierarchy_validation_event.root", "events")
streamed_event_correctness = get_event_correctness(stm_evt_validation)
standard_event_correctness = get_event_correctness(std_evt_validation)
plot_event_correctness(streamed_event_correctness, standard_event_correctness, ["streamed", "standard"], "folded")
plot_vertex_delta(stm_evt_validation.nu_vtx_dx, std_evt_validation.nu_vtx_dx, ["streamed", "standard"], "dx", "nu")
plot_vertex_delta(stm_evt_validation.nu_vtx_dy, std_evt_validation.nu_vtx_dy, ["streamed", "standard"], "dy", "nu")
plot_vertex_delta(stm_evt_validation.nu_vtx_dz, std_evt_validation.nu_vtx_dz, ["streamed", "standard"], "dz", "nu")
plot_vertex_delta(stm_evt_validation.nu_vtx_dr, std_evt_validation.nu_vtx_dr, ["streamed", "standard"], "dr", "nu")

# Misc cross checks

In [ ]:
def identify_broken(baseline, feature, file_id):
    idx_correct_base = np.where((baseline.file == file_id) & (baseline.n_good_tier1_matches == baseline.n_tier1_nodes))
    evt_correct_base = baseline.orig_event[idx_correct_base]
    idx_incorrect_feat = np.where((feature.file == file_id) & (feature.n_good_tier1_matches != feature.n_tier1_nodes))
    evt_incorrect_feat = feature.orig_event[idx_incorrect_feat]
    
    return np.intersect1d(evt_correct_base, evt_incorrect_feat)

def identify_fixed(baseline, feature, file_id):
    idx_incorrect_base = np.where((baseline.file == file_id) & (baseline.n_good_tier1_matches != baseline.n_tier1_nodes))
    evt_incorrect_base = baseline.orig_event[idx_incorrect_base]
    idx_correct_feat = np.where((feature.file == file_id) & (feature.n_good_tier1_matches == feature.n_tier1_nodes))
    evt_correct_feat = feature.orig_event[idx_correct_feat]
    
    return np.intersect1d(evt_incorrect_base, evt_correct_feat)


def identify_missing(baseline, feature, file_id):
    idx_base = np.where(baseline.file == file_id)
    evt_base = baseline.orig_event[idx_base]
    idx_feat = np.where(feature.file == file_id)
    evt_feat = feature.orig_event[idx_feat]
    
    return list(set(evt_base).difference(evt_feat))

In [ ]:
identify_broken(std_evt_validation, stm_evt_validation, 0)

In [ ]:
identify_fixed(std_evt_validation, stm_evt_validation, 0)

In [ ]:
broken = fixed = 0
files = np.unique(std_evt_validation.file)
for fid in files:
    broken += len(identify_broken(std_evt_validation, stm_evt_validation, fid))
    fixed += len(identify_fixed(std_evt_validation, stm_evt_validation, 0))
print(f"Broken: {broken} Fixed: {fixed} Events: {len(np.unique(std_evt_validation.event))}")

In [ ]:
summary_missing = {}
for fid in files:
    summary_missing[fid] = identify_missing(std_evt_validation, stm_evt_validation, fid)

In [ ]:
summary_missing

In [ ]:
missing_list = list(summary_missing.values())
flat_list = [item for sublist in missing_list for item in sublist]
len(flat_list)

In [ ]:
summary_broken = {}
for fid in files:
    summary_broken[fid] = identify_broken(std_evt_validation, stm_evt_validation, fid)

In [ ]:
summary_broken

In [ ]:
idx = np.where((std_evt_validation.file == 0) & (std_evt_validation.orig_event == 29))
print(std_evt_validation.n_good_tier1_matches[idx], std_evt_validation.n_tier1_nodes[idx])

In [ ]:
idx = np.where((stm_evt_validation.file == 0) & (stm_evt_validation.orig_event == 29))
print(stm_evt_validation.n_good_tier1_matches[idx], stm_evt_validation.n_tier1_nodes[idx])

In [ ]:
val_streamed.purity_list_w[0]

In [ ]:
val_streamed.completeness_list_w[0]

In [ ]:
val_streamed.mc_id[0]

In [ ]:
val_streamed.mc_pdg[0]

In [ ]:
val_streamed.purity_list_w[0] * val_streamed.completeness_list_w[0]

In [ ]:
val_streamed.purity_list_w

In [ ]:
val_streamed.completeness_list_w

In [ ]:
#This code returns specificed parameters of root files and includes some other attributes
file_energy = 14 #In MeV, which energy file should be read
file_criteria = "reco_nhits_list" #From list of keys above

full_criteria = eval("validation." + file_criteria)
size_criteria = np.size(full_criteria)
validation = MCValidation("hierarchy_validation_MC_" + str(file_energy) + "MeV.root", "events")

def make_linear(validation):
    temp_array = []
    for i in range(0, size_criteria - 1):
        subdivision_size = np.size(validation[i])
        x = 1
        temp_array.append(validation[i])
        if subdivision_size != 1:
            while x < subdivision_size:
                temp_array.append(validation[i][x])
                x += 1
    size = np.size(temp_array)
    return temp_array, size

#size_difference = int(make_linear(full_criteria)[1]) - int(size_criteria)
    

print("\033[1m" + file_criteria + "\033[0m" + " is as follows:")
print(full_criteria)
print("\n")
print("The length of " + file_criteria + " is " + "\033[1m" + str(size_criteria) + "\033[0m")
print("The number of empty events recorded " + "\033[1m" + str(abs(size_difference)) + "\033[0m" )